In [1]:
from Bio import Entrez
from Bio import Medline


In [2]:
Entrez.email = 'rsousa.works@gmail.com'

In [3]:
term = "HLA-DQA2" # Definimos o termo de pesquisa

In [4]:
# Realizamos uma Query Geral para pesquisar pelo termo para ter uma ideia de quantos resultados de pesquisa existem
handle = Entrez.egquery(term=term)
egq_res = Entrez.read(handle)


In [5]:
'''
Link para documentação:

(Documentação Subpackage Entrez)    https://biopython.org/docs/1.75/api/Bio.Entrez.html 
(Parâmetros do Entrez.egquery)      https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EGQuery

Neste caso vou-me focar na 'pubmed'.

'''

for _ in egq_res['eGQueryResult']:
    if 'pubmed' in _.values():
        print('Resultados em "pubmed":',_['Count'])

Resultados em "pubmed": 90


In [6]:
# Agora efetuamos um esearch para sacar uma lista dos 10 primeiros PMIDs para depois analisar os artigos

handle      = Entrez.esearch(db='pubmed',term=term, retmax=10)
esearch_res = Entrez.read(handle)

In [7]:
'''
Link para documentação:

(Parâmetros do Entrez.esearch)  http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch

Interessa-nos a lista de PMIDs para poder sacar os abstracts dos artigos e fazer a sua leitura.
Criamos uma lista com essa lista de IDs

'''

lista_ids = esearch_res['IdList']
lista_ids

['37950809', '37827449', '37679517', '37427097', '37254219', '37004177', '36973646', '36927638', '36793127', '36336987']

In [8]:
'''
Sacamos os títulos e os DOIs dos diferentes artigos - para referência futura

'''
handle   = Entrez.esummary(db='pubmed', id=','.join(lista_ids))
esum_res = Entrez.read(handle)


In [48]:
esum_res

[{'Item': [], 'Id': '37950809', 'PubDate': '2023 Nov 11', 'EPubDate': '2023 Nov 11', 'Source': 'Hepatol Int', 'AuthorList': ['Chen L', 'Zhao Y', 'Li M', 'Lv G'], 'LastAuthor': 'Lv G', 'Title': 'Proteome-wide Mendelian randomization highlights AIF1 and HLA-DQA2 as targets for primary sclerosing cholangitis.', 'Volume': '', 'Issue': '', 'Pages': '', 'LangList': ['English'], 'NlmUniqueID': '101304009', 'ISSN': '1936-0533', 'ESSN': '1936-0541', 'PubTypeList': ['Journal Article'], 'RecordStatus': 'PubMed - as supplied by publisher', 'PubStatus': 'aheadofprint', 'ArticleIds': {'medline': [], 'pubmed': ['37950809'], 'doi': '10.1007/s12072-023-10608-8', 'pii': '10.1007/s12072-023-10608-8'}, 'DOI': '10.1007/s12072-023-10608-8', 'History': {'medline': ['2023/11/11 20:51'], 'pubmed': ['2023/11/11 20:51'], 'received': '2023/07/18 00:00', 'accepted': '2023/10/09 00:00', 'entrez': '2023/11/11 11:08'}, 'References': [], 'HasAbstract': IntegerElement(1, attributes={}), 'PmcRefCount': IntegerElement(0,

In [49]:
# Usamos um fetch para ir buscar os abstracts dos artigos da lista.

handle = Entrez.efetch(db='pubmed',id=','.join(lista_ids),rettype='abstract',retmode='text')
record = handle.read()

ISTO PROVAVELMENTE É PARA IGNORAR.

Após analisar os abstracts , alguns que saltam à vista / falam de possíveis relações entre os genes (a refinar a pesquisa abaixo):

- Genome-wide identification of m(6)A-associated single nucleotide polymorphisms 
in complex diseases of nervous system.
Neurosci Lett. 2023 Nov 20;817:137513. doi: 10.1016/j.neulet.2023.137513. Epub
2023 Oct 10.PMID: 37827449 [Indexed for MEDLINE]

- Immune microenvironment analysis and novel biomarkers of early-stage lung 
adenocarcinoma evolution.
Front Oncol. 2023 Jun 23;13:1150098. doi: 10.3389/fonc.2023.1150098.
eCollection  2023.
PMID: 37427097


- Rheumatology (Oxford). 2023 Sep 1;62(9):3213-3218. doi: 
10.1093/rheumatology/kead152.
Stratified genetic analysis reveals sex differences in MPO-ANCA-associated 
vasculitis.
PMID: 37004177 [Indexed for MEDLINE]

O 8 e o 9


In [68]:
# Convertendo este processo numa função.

def lit_search(term,db = 'pubmed'):

    # TODO Adicionar indicações para fazer o print e elencar a lista de resultados

    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            print('Resultados em "pubmed":',_['Count'])

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmax=10)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    print('Top 10 artigos ->',lista_ids) # Converter para uma lista de títulos mais à frente
    
    

    # Sacar os abstracts
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype='abstract',retmode='text')
    fetch_res = handle.read()
    
    # Usar com print para já, para não estar a bagunçar demasiado.
    # Perceber como se pode implementar a escrita de ficheiros para ser mais fácil de mastigar os resultados.

    return fetch_res



In [70]:
termo = 'hla-dqa2 polymorphism'

teste = lit_search(termo)

Resultados em "pubmed": 50
Top 10 artigos -> ['37827449', '37679517', '36973646', '36269708', '35606880', '35568807', '35500836', '35436980', '34888651', '34095496']


In [71]:
print(teste)

1. Neurosci Lett. 2023 Nov 20;817:137513. doi: 10.1016/j.neulet.2023.137513. Epub
 2023 Oct 10.

Genome-wide identification of m(6)A-associated single nucleotide polymorphisms 
in complex diseases of nervous system.

Guo F(1), Kang J(1), Xu J(1), Wei S(2), Tao J(1), Dong Y(1), Ma Y(1), Tian H(2), 
Guo X(2), Bi S(2), Zhang C(2), Lv H(2), Shang Z(2), Jiang Y(3), Zhang M(4).

Author information:
(1)College of Bioinformatics Science and Technology, Harbin Medical University, 
Harbin, China; The EWAS Project, China.
(2)College of Bioinformatics Science and Technology, Harbin Medical University, 
Harbin, China.
(3)College of Bioinformatics Science and Technology, Harbin Medical University, 
Harbin, China; The EWAS Project, China. Electronic address: 
jiangyongshuai@hrbmu.edu.cn.
(4)College of Bioinformatics Science and Technology, Harbin Medical University, 
Harbin, China; The EWAS Project, China. Electronic address: 
zhangmingming@hrbmu.edu.cn.

N6-methyladenosine (m6A) is one of the most a